# 利用alpha来无风险套利

## 1. alpha如何创造套利机会

In [9]:
import numpy as np
import statsmodels.api as sm
import pandas as pd

In [10]:
np.random.seed(0) 

# 生成模拟市场指数收益
market_returns = np.random.normal(0.01, 0.05, 100)  # 假设市场平均每天有1%的收益，标准差5%

# 生成两个资产的returns数据
# 资产A有一些市场影响和一些alpha
A_returns = 0.1 + 1.2*market_returns + np.random.normal(0, 0.03, 100) 
# 这个0.1就可以理解为超额回报的部分，正是因为有它的存在创造出了无风险套利机会

# 资产B主要受市场影响，几乎没有alpha
B_returns = 0.01 + 1.0*market_returns + np.random.normal(0, 0.03, 100)

data = pd.DataFrame({
    'Market': market_returns,
    'Asset_A': A_returns,
    'Asset_B': B_returns
})

In [11]:
# 利用线性回归找到alpha
X = sm.add_constant(data['Market'])  # 添加常数项
y_A = data['Asset_A']

model_A = sm.OLS(y_A, X)
results_A = model_A.fit()
print("Asset A:")
print(results_A.summary())

y_B = data['Asset_B']
model_B = sm.OLS(y_B, X)
results_B = model_B.fit()
print("\nAsset B:")
print(results_B.summary())

Asset A:
                            OLS Regression Results                            
Dep. Variable:                Asset_A   R-squared:                       0.811
Model:                            OLS   Adj. R-squared:                  0.809
Method:                 Least Squares   F-statistic:                     421.1
Date:                Tue, 15 Oct 2024   Prob (F-statistic):           2.96e-37
Time:                        17:15:22   Log-Likelihood:                 205.98
No. Observations:                 100   AIC:                            -408.0
Df Residuals:                      98   BIC:                            -402.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1016      0.003     31.565

In [12]:
# 策略实施：每天做多1单位的资产A，做空1.2单位的资产B
# 假设我们投资100天
portfolio_returns = results_A.params['const'] - 1.2 * results_B.params['const']

print(f"预期每日无风险套利收益: {portfolio_returns*100:.2f}%")

预期每日无风险套利收益: 9.24%
